In [ ]:
import numpy as np
import pydensecrf.densecrf as dcrf
from skimage.io import imread, imsave
from pydensecrf.utils import unary_from_labels, create_pairwise_bilateral
from skimage.color import gray2rgb
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
%matplotlib inline

import webcolors
from matplotlib.patches import Patch
from skimage.transform import resize

In [ ]:
def decode(PredictionString):
    mask = PredictionString.split()
    image = np.zeros(256 * 256, dtype=np.uint8)
    for idx, value in enumerate(mask):
        image[idx] = int(value)
    # image = image.reshape(256, 256)
    return image.reshape(256, 256)

In [ ]:
# 파일 넣기
df = pd.read_csv('./SY.csv') # <<<<<

plt.figure(figsize=(30, 15))
plt.subplots_adjust(bottom=0.2, top=0.8, hspace=0.2)

for i in range(5):
    decoded_mask = decode(df.loc[i, 'PredictionString'])
    plt.subplot(1, 6, i+1)
    plt.imshow(decoded_mask)
    plt.title(df.loc[i, 'image_id'])

In [ ]:
# class_colormap = pd.read_csv("/opt/ml/input/code/class_dict.csv")
class_colormap = [
    [0, 0, 0], [192, 0, 128], [0, 128, 192], [0, 128, 64], [128, 0, 0], [64, 0, 128],
    [64, 0, 192], [192, 128, 64], [192, 192, 128], [64, 64, 128], [128, 0, 192]
]
classes = ['Backgroud', 'General trash', 'Paper', 'Paper pack', 'Metal',
            'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing']


In [ ]:
test_path = '/opt/ml/input/data/'

In [ ]:
def create_trash_label_colormap():
    """Creates a label colormap used in Trash segmentation.
    Returns:
        A colormap for visualizing segmentation results.
    """
    colormap = np.zeros((11, 3), dtype=np.uint8)
    for inex, (r, g, b) in enumerate(class_colormap):
        colormap[inex] = [r, g, b]
    
    return colormap

def label_to_color_image(label):
    """Adds color defined by the dataset colormap to the label.

    Args:
        label: A 2D array with integer type, storing the segmentation label.

    Returns:
        result: A 2D array with floating type. The element of the array
                is the color indexed by the corresponding element in the input label
                to the trash color map.

    Raises:
        ValueError: If label is not of rank 2 or its value is larger than color
              map maximum entry.
    """
    if label.ndim != 2:
        raise ValueError('Expect 2-D input label')

    colormap = create_trash_label_colormap()

    if np.max(label) >= len(colormap):
        raise ValueError('label value too large.')

    return colormap[label]

In [ ]:
def crf(original_image, mask_img):
    
    labels = mask_img.flatten()

    n_labels = 11
    
    #Setting up the CRF model
    # d = dcrf.DenseCRF2D(original_image.shape[1], original_image.shape[0], n_labels)
    d = dcrf.DenseCRF2D(original_image.shape[1], original_image.shape[0], n_labels)

    # get unary potentials (neg log probability)
    U = unary_from_labels(labels, n_labels, gt_prob=0.7, zero_unsure=False)
    d.setUnaryEnergy(U)

    # This adds the color-independent term, features are the locations only.
    d.addPairwiseGaussian(sxy=(3, 3), compat=3, kernel=dcrf.DIAG_KERNEL,
                      normalization=dcrf.NORMALIZE_SYMMETRIC)

    d.addPairwiseBilateral(sxy=(10, 10), srgb=50, rgbim=original_image, compat=5)

    #Run Inference for 10 steps 
    Q = d.inference(40)

    # Find out the most probable class for each pixel.
    MAP = np.argmax(Q, axis=0)

    # return MAP.reshape((original_image.shape[0],original_image.shape[1]))
    return MAP.reshape((original_image.shape[0],original_image.shape[1]))

"""
visualizing the effect of applying CRF

"""

category_and_rgb = [[classes[idx], (r,g,b)] for idx, (r, g, b) in enumerate(class_colormap)]
legend_elements = [Patch(facecolor=webcolors.rgb_to_hex(rgb), edgecolor=webcolors.rgb_to_hex(rgb), label=category) for category, rgb in category_and_rgb]
     

nImgs = 1
i = np.random.randint(len(df))
# i = 7
j = 1

plt.figure(figsize=(20,20))
plt.subplots_adjust(wspace=0.2,hspace=0.1)  #adjust this to change vertical and horiz. spacings..

while True:
    if str(df.loc[i,'PredictionString'])!=str(np.nan):        
        # print(df.loc[i,'image_id'])
        decoded_mask = decode(df.loc[i,'PredictionString'])      
        orig_img = imread(test_path+df.loc[i,'image_id'])
        orig_img = resize(orig_img, (256, 256, 3))
        orig_img = np.uint8(255 * orig_img)
        #Applying CRF on FCN-16 annotated image
        crf_output = crf(orig_img,decoded_mask)
        plt.subplot(nImgs,4,4*j-3)
        plt.imshow(orig_img)
        plt.title('Original image')
        plt.title(df.loc[i,'image_id'])
        plt.subplot(nImgs,4,4*j-2)
        plt.imshow(label_to_color_image(decoded_mask)) 
        plt.title(df.loc[i,'image_id'])
        plt.title('Original Mask')
        plt.subplot(nImgs,4,4*j-1)
        plt.imshow(label_to_color_image(crf_output)) 
        plt.title('Mask after CRF')
        plt.legend(handles=legend_elements, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0)
        if j == nImgs:
            break
        else:
            j = j + 1
    i = i + 1

In [ ]:
def encode(im):
    '''
    im: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = im.flatten()
    return ' '.join(str(x) for x in pixels)

In [ ]:
for i in tqdm(range(df.shape[0])):
    if str(df.loc[i,'PredictionString'])!=str(np.nan):        
        decoded_mask = decode(df.loc[i,'PredictionString'])        
        orig_img = imread(test_path+df.loc[i,'image_id'])
        orig_img = resize(orig_img, (256, 256, 3))
        orig_img = np.uint8(255 * orig_img)  
        crf_output = crf(orig_img,decoded_mask)
        df.loc[i,'PredictionString'] = encode(crf_output)

In [ ]:
df.to_csv('swin_large_del_Last_best59t_crf.csv',index=False)